In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-22 15:02:01,883 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2082546278608) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2082546281200) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2082546282352) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2082546280192) was called 1 times
Running: Function "load_module_lookup" (id=2082546281920) was called 1 times
Running: Function "load_ID_lookup" (id=2082546280624) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=15)

Time shifted by 15Minutes


In [77]:
#fa_sel = feat.get_data_faults(fa_floor, modules=[1],PTT = 'PTT011')

#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_floor,fault_agg_level= 'Asset Code', agg_type = 'count')

In [78]:
#av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = {'Pick Station' : ['PTT011']}, remove_high_AT = True, AT_limit = None)

av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Quadrant')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Quadrant')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2082546280048) was called 7 times


In [79]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'Quadrant')

In [82]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=y

In [62]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'threshold': -np.inf,
                    'max_features': 180}

X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

X_sel_XGB = md.select_features(X, y, XGB_mdl, **selection_options)

X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       'random_state': None})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X_sel_rf, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

Feature Importance Ranking: 
 
    Importance Feature
0    0.748786   TOTES
1    0.048907       0
2    0.016103    3416
3    0.015164    3611
4    0.011572    3428
5    0.010301    5105
6    0.008197    5120
7    0.008120    3420
8    0.007439    3430
9    0.007045    5329

Regression Metrics: 
 
           RF Metrics
MAE         0.036888
MSE         0.002225
RMSE        0.047168
MAPE%      10.390000
ACC%       89.610000
OOB         0.700031
R2_Train    0.959320
R2_Pred     0.755573 

Feature Importance Ranking: 
 
    Importance Feature
0    0.292945   TOTES
1    0.065436   12015
2    0.052193    5102
3    0.042765    8270
4    0.028123    9950
5    0.022314    5115
6    0.021418    6207
7    0.021141    5096
8    0.020384   11976
9    0.018995    5577

Regression Metrics: 
 
          XGB Metrics
MAE        0.0397652
MSE       0.00261956
RMSE       0.0511816
MAPE%          11.37
ACC%           88.63
OOB              N/A
R2_Train    0.999959
R2_Pred     0.712206 


Number of Selected 

In [90]:
df

,timestamp,Downtime,Blue Tote Loss,Grey Tote Loss,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,...,SCSM12,SCSM13,SCSM14,SCSM15,SCSM17,SCSM18,SCSM19,SCSM20,SCSM22,TOTES
0,2020-11-20 00:00:00,0.183683,0.183683,0.000000,0,0,0,0,0,0,...,2,0,0,0,0,8,0,2,6,36.271186
1,2020-11-20 00:00:00,0.304172,0.068667,0.235506,0,0,0,0,0,0,...,2,0,0,0,0,8,0,2,6,38.745763
2,2020-11-20 00:00:00,0.269838,0.000000,0.269838,0,0,0,0,0,0,...,2,0,0,0,0,8,0,2,6,83.648148
3,2020-11-20 00:00:00,0.148706,0.066667,0.082040,0,0,0,0,0,0,...,2,0,0,0,0,8,0,2,6,61.052632
4,2020-11-20 00:00:00,0.167554,0.022667,0.144887,0,0,0,0,0,0,...,2,0,0,0,0,8,0,2,6,59.152542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8413,2020-12-09 17:00:00,0.706404,0.502667,0.203738,0,0,0,0,0,0,...,2,0,2,2,0,17,6,0,0,26.843137
8414,2020-12-09 17:00:00,0.356481,0.285333,0.071147,0,0,0,0,0,0,...,2,0,2,2,0,17,6,0,0,41.533333
8415,2020-12-09 17:00:00,0.192146,0.002000,0.190146,0,0,0,0,0,0,...,2,0,2,2,0,17,6,0,0,73.516667
8416,2020-12-09 17:00:00,0.268037,0.056667,0.211370,0,0,0,0,0,0,...,2,0,2,2,0,17,6,0,0,56.033333


### Add LM - metrics

In [93]:
y=1-y
model = sm.OLS(y, X)
results = model.fit()

keep_features = results.pvalues[results.pvalues < 0.10].index
model = sm.OLS(y,X[keep_features])
results = model.fit()

keep_features = results.pvalues[results.pvalues < 0.10].index
model = sm.OLS(y,X[keep_features])
results = model.fit()

keep_features = results.pvalues[results.pvalues < 0.10].index
model = sm.OLS(y,X[keep_features])
results = model.fit()

print(len(keep_features))

cv_R2 = md.cross_validate_r2(Linear_mdl, X[keep_features], y)

print(results.summary())

#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


258

Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.488290
2       0.446121
3       0.415895
4       0.390991
5       0.475142
6       0.501414
7       0.425913
8       0.444732
9       0.386106
10      0.402076
Mean    0.437668
STD     0.038559
                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.899
Model:                            OLS   Adj. R-squared (uncentered):              0.896
Method:                 Least Squares   F-statistic:                              292.8
Date:                Tue, 22 Dec 2020   Prob (F-statistic):                        0.00
Time:                        17:03:19   Log-Likelihood:                          5294.6
No. Observations:                8418   AIC:                                 -1.009e+04
Df Residuals:                    8170   BIC:                                     -8348.
Df Model:             

In [17]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

C0801STA002    335
C0803STA112    289
C0703STA102     92
C1401ACH001     90
C0503STA251     87
C0501STA012     76
C0503STA252     73
C0502STA056     62
C0801ACH001     61
C0704STA201     47
C0903ACH113     33
C0903STA112     29
C0901STA012     28
C1003STA251     27
C1101ACH001     24
C0503ACH234     24
C1303STA102     17
C1403ACH113     13
C1103STA102     13
C1101STA012     12
C1403STA112     10
C1003STA252      9
C1301STA002      8
C1401ACH011      8
C0603STA102      7
C0904STA206      7
C0901ACH011      7
C1203ACH106      6
C0503RDC101      6
C0701STA002      6
C1201STA012      5
C0501STA002      5
C0703ACH101      5
C0704STA206      4
C1001STA002      4
C0503ACH233      4
C0801ACH011      4
C1201ACH001      4
C0803STA102      3
C1101STA002      3
C1403STA102      3
C0901RDC017      2
C1203ACH113      2
C0901STA015      2
C1401STA012      2
C0701ACH011      2
C1003ACH235      1
C0904BCL208      1
C0502STA057      1
C1003ACH232      1
C1201ACH011      1
C1101ACH011      1
C0803ACH106 

In [70]:
fa[fa['Fault ID']==7647]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
165704,441848.0,SCS M04: single module 2 access door open fron...,2020-11-25 05:28:29,SCSM04,SCSM04,17.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165737,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 21:08:53,SCSM04,SCSM04,18.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165719,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 14:26:23,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165720,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 13:03:40,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165739,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 17:49:05,SCSM04,SCSM04,21.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165721,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 12:36:34,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165714,441848.0,SCS M04: single module 2 access door open fron...,2020-11-29 22:39:41,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165729,441848.0,SCS M04: single module 2 access door open fron...,2020-12-06 21:49:15,SCSM04,SCSM04,23.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165717,441848.0,SCS M04: single module 2 access door open fron...,2020-12-03 08:02:11,SCSM04,SCSM04,24.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165736,441848.0,SCS M04: single module 2 access door open fron...,2020-12-05 04:45:00,SCSM04,SCSM04,26.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault


In [19]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

NameError: name 'fa_sel' is not defined

In [ ]:
X

In [ ]:
y

In [ ]:
at_agg.loc["2020-11-20":].head(50)